# Cabin

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
le = LabelEncoder()

<IPython.core.display.Javascript object>

In [4]:
df = pd.read_csv("../../data/titanic/prepare.csv").set_index("PassengerId")
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,HasNamesakes,Title,FamilySize,IsAlone,TicketSeries,SurnameSeries,TitleSeries,IsTest,CabinSeries,isCabinSeriesPreds
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,-0.503595,NaN,...,0,Mr,2,0,0,100,2,0,5,1
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,0.734503,C85,...,0,Mrs,2,0,12,182,3,0,2,0
3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,-0.490544,NaN,...,0,Miss,1,1,21,329,1,0,5,1
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,0.382925,C123,...,0,Mrs,2,0,26,267,3,0,2,0
5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,-0.488127,NaN,...,0,Mr,1,1,26,15,2,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,-0.488127,NaN,...,0,Mr,1,1,2,753,2,1,5,1
1306,NaN,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,1.461829,C105,...,0,Other,1,1,12,593,4,1,2,0
1307,NaN,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,-0.503595,NaN,...,0,Mr,1,1,19,699,2,1,4,1


<IPython.core.display.Javascript object>

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Survived            891 non-null    float64
 1   Pclass              1309 non-null   int64  
 2   Name                1309 non-null   object 
 3   Sex                 1309 non-null   int64  
 4   Age                 1046 non-null   float64
 5   SibSp               1309 non-null   int64  
 6   Parch               1309 non-null   int64  
 7   Ticket              1309 non-null   object 
 8   Fare                1309 non-null   float64
 9   Cabin               295 non-null    object 
 10  Embarked            1309 non-null   int64  
 11  Surname             1309 non-null   object 
 12  HasNamesakes        1309 non-null   int64  
 13  Title               1309 non-null   object 
 14  FamilySize          1309 non-null   int64  
 15  IsAlone             1309 non-null   int64  
 16  Ticket

<IPython.core.display.Javascript object>

In [6]:
X = df[
    [
        "Pclass",
        "Sex",
        "SibSp",
        "Parch",
        "Fare",
        "Embarked",
        "HasNamesakes",
        "FamilySize",
        "IsAlone",
        "TicketSeries",
        "SurnameSeries",
        "TitleSeries",
    ]
]
X

,Pclass,Sex,SibSp,Parch,Fare,Embarked,HasNamesakes,FamilySize,IsAlone,TicketSeries,SurnameSeries,TitleSeries
PassengerId,,,,,,,,,,,,
1,3,1,1,0,-0.503595,2,0,2,0,0,100,2
2,1,0,1,0,0.734503,0,0,2,0,12,182,3
3,3,0,0,0,-0.490544,2,0,1,1,21,329,1
4,1,0,1,0,0.382925,2,0,2,0,26,267,3
5,3,1,0,0,-0.488127,2,0,1,1,26,15,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,0,0,-0.488127,2,0,1,1,2,753,2
1306,1,0,0,0,1.461829,0,0,1,1,12,593,4
1307,3,1,0,0,-0.503595,2,0,1,1,19,699,2


<IPython.core.display.Javascript object>

In [7]:
train_index = df[~df["Cabin"].isna()].index
train_index

Int64Index([   2,    4,    7,   11,   12,   22,   24,   28,   32,   53,
            ...
            1282, 1283, 1287, 1289, 1292, 1296, 1297, 1299, 1303, 1306],
           dtype='int64', name='PassengerId', length=295)

<IPython.core.display.Javascript object>

In [8]:
X_train = X[X.index.isin(train_index)]
X_test = X[~X.index.isin(train_index)]

X_train.shape, X_test.shape

((295, 12), (1014, 12))

<IPython.core.display.Javascript object>

## Prepare `y`

In [9]:
df["CabinSeries"] = df["Cabin"].str[0]

df["CabinSeries"] = df["CabinSeries"].astype("category").cat.codes
df.loc[df["CabinSeries"] == -1, "CabinSeries"] = np.nan

df["CabinSeries"].value_counts(dropna=False)

NaN    1014
2.0      94
1.0      65
3.0      46
4.0      41
0.0      22
5.0      21
6.0       5
7.0       1
Name: CabinSeries, dtype: int64

<IPython.core.display.Javascript object>

In [10]:
y_train = df[df.index.isin(train_index)][["CabinSeries"]]
y_train

,CabinSeries
PassengerId,
2,2.0
4,2.0
7,4.0
11,6.0
12,2.0
...,...
1296,3.0
1297,3.0
1299,2.0


<IPython.core.display.Javascript object>

# Train

In [11]:
model = CatBoostClassifier(loss_function="MultiClass")

<IPython.core.display.Javascript object>

In [12]:
model.fit(Pool(X_train, y_train), verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

<IPython.core.display.Javascript object>

# Save

In [13]:
y_preds = model.predict(X_test).ravel()
y_preds

array([5., 4., 5., ..., 4., 5., 5.])

<IPython.core.display.Javascript object>

In [14]:
y_test = pd.DataFrame({"PassengerId": X_test.index, "CabinSeries": y_preds}).set_index(
    "PassengerId"
)
y_test

,CabinSeries
PassengerId,
1,5.0
3,4.0
5,5.0
6,5.0
8,5.0
...,...
1304,5.0
1305,5.0
1307,4.0


<IPython.core.display.Javascript object>

In [15]:
df["isCabinSeriesPreds"] = df["CabinSeries"].isna().astype(int)

<IPython.core.display.Javascript object>

In [16]:
df.fillna(y_test, inplace=True)
df["CabinSeries"] = df["CabinSeries"].astype(int)

<IPython.core.display.Javascript object>

In [17]:
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,HasNamesakes,Title,FamilySize,IsAlone,TicketSeries,SurnameSeries,TitleSeries,IsTest,CabinSeries,isCabinSeriesPreds
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,-0.503595,NaN,...,0,Mr,2,0,0,100,2,0,5,1
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,0.734503,C85,...,0,Mrs,2,0,12,182,3,0,2,0
3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,-0.490544,NaN,...,0,Miss,1,1,21,329,1,0,4,1
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,0.382925,C123,...,0,Mrs,2,0,26,267,3,0,2,0
5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,-0.488127,NaN,...,0,Mr,1,1,26,15,2,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,-0.488127,NaN,...,0,Mr,1,1,2,753,2,1,5,1
1306,NaN,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,1.461829,C105,...,0,Other,1,1,12,593,4,1,2,0
1307,NaN,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,-0.503595,NaN,...,0,Mr,1,1,19,699,2,1,4,1


<IPython.core.display.Javascript object>

In [18]:
df.to_csv("../../data/titanic/prepare.csv")

<IPython.core.display.Javascript object>